In [ ]:
import os
import scanpy as sc


base_dir = "/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/revision/individual_samples/mtx/"
h5ad_dir = "/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/revision/individual_samples/h5ad/"
#adatas = {}

In [ ]:
# get metadata
import pandas as pd

di_table = pd.read_csv("/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/revision/sample_clin_data/sample_table_DI.txt", sep="\t")


In [ ]:
di_table

In [ ]:
d1 = dict(zip(di_table['Sample'], di_table['DI_Sample']))
d2 = dict(zip(di_table['Sample'], di_table['UPN']))
d3 = dict(zip(di_table['Sample'], di_table['DI_UPN']))
d4 = dict(zip(di_table['Sample'], di_table['Collection']))

In [ ]:
# iterate through folders
for sample in os.listdir(base_dir):
    if not sample or not sample[0].isalnum():
        continue
        
    sample_path = os.path.join(base_dir, sample) 
        
    mtx = os.path.join(sample_path, "counts.mtx")
    genes = os.path.join(sample_path, "genes.tsv")
    barcodes = os.path.join(sample_path, "barcodes.tsv")
    meta = pd.read_csv(os.path.join(sample_path, "metadata.tsv"), sep=" ")
    
    # read matrix and transpose (Scanpy reads mtx as cell×gene)
    adata = sc.read_mtx(mtx).T
    adata.var_names = [x.strip() for x in open(genes)]
    adata.obs_names = [x.strip() for x in open(barcodes)]

    # update metadata
    meta_add = meta[['nCount_RNA', 'nFeature_RNA', 'percent.mt', 'rho', 'predicted_doublet', 'doublet_score', 'S.Score', 'G2M.Score', 'Phase']]
    adata.obs = adata.obs.join(meta_add, how="left")
    
    adata.obs['Sample'] = sample
    adata.obs['DI_Sample'] = d1[sample]
    adata.obs['UPN'] = d2[sample]
    adata.obs['DI_UPN'] = d3[sample]
    adata.obs['Collection'] = d4[sample]
    
    # save to h5ad
    out_path = os.path.join(h5ad_dir, f"{sample}.h5ad")
    adata.write(out_path)
    print(f"Saved {sample} to {out_path}")